<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/RedditConversationQuality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Reddit Data  

#Pushshift API

##Write submmisons to file

In [12]:
import json

def filter_json_objects(data, keys):
    filtered_data = []
    for item in data:
        filtered_item = {}
        for key in keys:
            if key in item:
                filtered_item[key] = item[key]
        filtered_data.append(filtered_item)
    return filtered_data

In [14]:
import requests
import datetime
import json

# define the date range
start_date = datetime.datetime(2023, 3, 1)
end_date = datetime.datetime(2023, 3, 31)
keys = ['created_utc', 'id']

# read the last UTC timestamp from file
try:
    with open("last_timestamp.txt", "r") as f:
        last_timestamp = int(f.read())
except FileNotFoundError:
    last_timestamp = int(start_date.timestamp())

# define the API endpoint
# fields not working
url = f"https://api.pushshift.io/reddit/search/submission/?sort=created_utc&size=100&after={int(start_date.timestamp())}&before={int(end_date.timestamp())}"

# make the API request and get the first set of submissions
response = requests.get(url)
submissions = filter_json_objects(response.json()["data"], keys)
print(response.json())

# write the submissions to a file as JSON
with open("submissions.json", "a") as f:
    for submission in submissions:
        json.dump(submission, f)
        f.write("\n")

# save the UTC timestamp of the last submission to file
last_submission = submissions[-1]
last_timestamp = last_submission["created_utc"]
with open("last_timestamp.txt", "w") as f:
    f.write(str(last_timestamp))

# get the next set of submissions using pagination
print("len ",len(submissions))
start = 0
while len(submissions) == 100:
    print("next")
    last_submission = submissions[-1]
    last_submission_date = datetime.datetime.fromtimestamp(last_submission["created_utc"])
    new_url = f"https://api.pushshift.io/reddit/search/submission/?size=100&after={int(last_submission_date.timestamp())}&before={int(end_date.timestamp())}"
    try:
        new_response = requests.get(new_url)
        new_submissions = filter_json_objects(new_response.json()["data"], keys)
        submissions = new_submissions
        with open("submissions.json", "a") as f:
            for submission in submissions:
                json.dump(submission, f)
                f.write("\n")
        # save the UTC timestamp of the last submission to file
        last_submission = submissions[-1]
        last_timestamp = last_submission["created_utc"]
        with open("last_timestamp.txt", "w") as f:
            f.write(str(last_timestamp))

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 429:
            retry_after = int(e.response.headers.get("Retry-After"))
            print(f"Got HTTP error 429, waiting {retry_after} seconds and retrying...")
            time.sleep(retry_after)
            continue
        else:
            raise e

{'data': [{'subreddit': 'phubpornWeb', 'selftext': '', 'author_fullname': 't2_w8y8o8jb', 'gilded': 0, 'title': 'Something you want to suck on?', 'link_flair_richtext': [], 'subreddit_name_prefixed': 'r/phubpornWeb', 'hidden': False, 'pwls': None, 'link_flair_css_class': None, 'thumbnail_height': 70, 'top_awarded_type': None, 'hide_score': True, 'quarantine': False, 'link_flair_text_color': 'dark', 'upvote_ratio': 1.0, 'author_flair_background_color': None, 'subreddit_type': 'public', 'total_awards_received': 0, 'media_embed': {}, 'thumbnail_width': 140, 'author_flair_template_id': None, 'is_original_content': False, 'secure_media': None, 'is_reddit_media_domain': False, 'is_meta': False, 'category': None, 'secure_media_embed': {}, 'link_flair_text': None, 'score': 1, 'is_created_from_ads_ui': False, 'author_premium': False, 'thumbnail': 'nsfw', 'edited': False, 'author_flair_css_class': None, 'author_flair_richtext': [], 'gildings': {}, 'post_hint': 'image', 'content_categories': None,

KeyboardInterrupt: ignored

##Write sample submission fields to file

In [ ]:
import requests
import json

# define the API endpoint with the sample submission ID
submission_id = "1273r9g"
url = f"https://api.pushshift.io/reddit/submission/search/?ids={submission_id}"

# make the API request and get the submission
response = requests.get(url)
submission_list = response.json()["data"]

# get the first submission from the list
submission = submission_list[0]

# write the submission keys to a file
with open("submission_keys.txt", "w") as f:
    for key in submission.keys():
        f.write(key + "\n")

##Write comments to file

###With Pushshift date search

###With comment_ids Pushshift

In [ ]:
import requests
import json

# read submission IDs from file
with open("submissions.json", "r") as f:
    submissions = [json.loads(line) for line in f]

# define the API endpoint with the submission IDs
url_template = "https://api.pushshift.io/reddit/submission/comment_ids/{}"

# make the API request and get the comments
comments_data = []
for submission in submissions:
    submission_id = submission["id"]
    url = url_template.format(submission_id)
    print(url)
    response = requests.get(url)
    comment_ids = response.json()
    comment_url = f"https://api.pushshift.io/reddit/comment/search/?ids={','.join(comment_ids)}"
    comment_response = requests.get(comment_url)
    comment_data = comment_response.json()["data"]
    comments_data.extend(comment_data)
    with open("comments_data.json", "a") as f:
        json.dump(comment_data, f)
#currently not working returns empty lists

https://api.pushshift.io/reddit/submission/comment_ids/1273r9i
https://api.pushshift.io/reddit/submission/comment_ids/1273r9j
https://api.pushshift.io/reddit/submission/comment_ids/1273r9n
https://api.pushshift.io/reddit/submission/comment_ids/1273r9p


KeyboardInterrupt: ignored

###With comment link-id

In [ ]:
import requests
import json

# read submission IDs from file
with open("submissions.json", "r") as f:
    submissions = [json.loads(line) for line in f]

# define the API endpoint with the submission IDs
url = "https://api.pushshift.io/reddit/comment/search/"

# make the API request and get the comments
comments_data = []
for submission in submissions:
    submission_id = submission["id"]
    print(submission["num_comments"])
    params = {"link_id": submission_id}
    response = requests.get(url, params=params)
    comments_data.extend(response.json()["data"])
    print(response.json()["data"])

# write the comments data to a file
with open("comments_data.json", "w") as f:
    json.dump(comments_data, f)
#Can not search by link_id

0
[]
1
[]
0


KeyboardInterrupt: ignored

#Async PRAW

In [ ]:
!pip install asyncpraw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.5/196.5 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 4.0 MB/s eta 0:00:00


In [ ]:
import asyncpraw
import datetime

# create an async Reddit instance
reddit = asyncpraw.Reddit(
    client_id='4Fh9K7U29aRt6GUpOkoqfw',
    client_secret='tyuF_v56l2bFmSqUOXFldWPtYsFViA',
    user_agent='Simurgh__'
)

In [ ]:
# define the subreddit and date range
subreddit = await reddit.subreddit("AmItheAsshole")
await subreddit.load()  # fetch the subreddit object from Reddit
start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2022, 1, 3)
# set up pagination
params = {"before": int(end_date.timestamp()), "after": int(start_date.timestamp()), "limit": 100}
print("start ",str(start_date.timestamp()))
print("end ", str(end_date.timestamp()))
start = 0
while start<1:
  submissions = subreddit.search("*", params=params)
  print("here")
  # save submissions to a file
  with open("submissions.txt", "w") as f:
    async for submission in submissions:
        f.write(str(submission.created_utc) + "\n")
  start += 1

start  1640995200.0
end  1641168000.0
here


#PRAW

##Installation

In [ ]:
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Imports

In [ ]:
import praw

##Hyper Parameters

In [ ]:
ALL_SUBREDDIT = 'all'
SAMPLE_SUBREDDIT = 'MachineLearning'

##Creating Reddit Instance

In [ ]:
reddit = praw.Reddit(client_id='4Fh9K7U29aRt6GUpOkoqfw',
                     client_secret='tyuF_v56l2bFmSqUOXFldWPtYsFViA',
                     user_agent='Simurgh__')

##Getting Posts

In [ ]:
# get 10 hot posts from the MachineLearning subreddit
hot_posts = reddit.subreddit(ALL_SUBREDDIT).hot(limit=10)

In [ ]:
for post in hot_posts:
    print(post.title)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Building a hobby-shelter while camping in Kelowna
meirl
A former bonded child labourer from Pakistan escaped the carpet factory at the age of 10, helped free over 3000 kids from slavery, helped bring down Pakistani carpet exports by $34 million, before being murdered at the age of 13
Judge Frank Caprio settles 250$ worth of fines and gives 25$ to pay for guys Uber who had 92¢ in his account and walked 5 miles just so he could make it to court
Boomer Comic
Man with An American flag shirt, reaction to kids protesting gun violence in their schools.
Soap bubble freezing
Facebook and Google are handing over user data to help police prosecute abortion seekers
Gotta burst your bubble, guys. We're all getting replaced soon.
How to massage your Eagle.


In [ ]:
# get MachineLearning subreddit data
ml_subreddit = reddit.subreddit('MachineLearning')

print(ml_subreddit.description)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



**[Rules For Posts](https://www.reddit.com/r/MachineLearning/about/rules/)**
--------
+[Research](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AResearch)
--------
+[Discussion](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ADiscussion)
--------
+[Project](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AProject)
--------
+[News](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ANews)
--------
***[@slashML on Twitter](https://twitter.com/slashML)***
--------
***[Chat with us on Slack](https://join.slack.com/t/rml-talk/shared_invite/enQtNjkyMzI3NjA2NTY2LWY0ZmRjZjNhYjI5NzYwM2Y0YzZhZWNiODQ3ZGFjYmI2NTU3YjE1ZDU5MzM2ZTQ4ZGJmOTFmNWVkMzFiMzVhYjg)***
--------
**Beginners:**
--------
Please have a look at [our FAQ and Link-Collection](http://www.reddit.com/r/MachineLearning/wiki/index)

[Metacademy](http://www.metacademy.org) is a great resource which compiles le

Because we only have a limited amoung of requests per day it is a good idea to save the scraped data in some kind of variable or file.

In [ ]:
import pandas as pd

posts = []
ml_subreddit = reddit.subreddit('MachineLearning')
for post in ml_subreddit.hot(limit=10):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



,title,score,id,subreddit,url,num_comments,body,created
0,[D] Simple Questions Thread,10,11ckopj,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,100,Please post your questions here instead of cre...,1.677427e+09
1,[D] Video 2 Minecraft — Comparing ControlNet a...,434,11igrea,MachineLearning,https://v.redd.it/ci3jjyo42tla1,7,,1.677972e+09
2,[D] Productization of deep learning models - w...,6,11isomm,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,4,I see so many new services built with deep lea...,1.678008e+09
3,[P] LazyShell - GPT based autocomplete for zsh,708,11hscl1,MachineLearning,https://i.redd.it/amnowgji6ola1.gif,51,,1.677913e+09
4,[D] Building an Open-Source LLM Provider for S...,9,11iooio,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,2,Hi guys! \n\nI've been thinking that as of now...,1.677994e+09
5,[D] LLaMA Model Parallelization and Server Con...,3,11iul6f,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,2,"Hey everyone,\n\nFirst of all, tldr at bottom,...",1.678015e+09
6,[D] First glance at LLaMA,51,11ibm1j,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,10,[https://medium.com/@enryu9000/mini-post-first...,1.677960e+09
7,[P] diffground - A simplistic Android UI to ac...,26,11idjj6,MachineLearning,https://i.redd.it/ckkqkjyrgsla1.png,5,,1.677965e+09
8,[P] A mobile-friendly way to edit photos with ...,3,11is9ge,MachineLearning,https://v.redd.it/c3avxdjswvla1,1,,1.678006e+09
9,[D] Is it possible to run Meta's LLaMA 65B mod...,41,11i4olx,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,44,I think it's clear that a single beefy rig cou...,1.677948e+09


In [ ]:
posts.to_csv('top_ml_subreddit_posts.csv')

##Getting Posts Comments

PRAW also allows us to get information about a specifc post/submission

In [ ]:
submission = reddit.submission(url="https://www.reddit.com/r/MapPorn/comments/a3p0uq/an_image_of_gps_tracking_of_multiple_wolves_in/")
# or 
submission = reddit.submission(id="a3p0uq") #id comes after comments/

This will work for some submission, but for others that have more comments this code will throw an AttributeError saying:

``AttributeError: 'MoreComments' object has no attribute 'body'``

These MoreComments object represent the “load more comments” and “continue this thread” links encountered on the websites, as described in more detail in the comment documentation.

There get rid of the MoreComments objects, we can check the datatype of each comment before printing the body.

The below cell is another way of getting rid of the MoreComments objects

The above codeblocks only got the top lebel comments. If we want to get the complete ``CommentForest`` we need to use the ``.list`` method.

In [ ]:
submission.comments.replace_more(limit=None)
i = 0
len(submission.comments.list())
#for comment in submission.comments.list():
    #print("parent", comment.parent_id)
    #print(comment.id)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

1015